# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [102]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [123]:
df =pd.read_csv('tic-tac-toe.csv')
df.shape

(958, 10)

In [74]:
working_columns = ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']
target = ['class']
all_cols = working_columns + target


In [75]:
categorial_features_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='constant',
                                                                           fill_value='missing')),
                                                 ('onehot',OneHotEncoder(handle_unknown='ignore'))])

preprocesor = ColumnTransformer(transformers=[('categorical_transformer',categorial_features_transformer,working_colums)])

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [76]:
import tensorflow as tf

In [77]:
train,test = train_test_split(df)

In [78]:
x_train, y_train = train[working_columns],train[target]
x_test, y_test = test[working_columns],test[target]

In [79]:
print(f'TRAIN SIZE: {x_train.shape}// {y_train.shape}')
print(f'TEST SIZE: {x_test.shape} // {y_test.shape}')

TRAIN SIZE: (718, 9)// (718, 1)
TEST SIZE: (240, 9) // (240, 1)


In [80]:
x_train = preprocesor.fit_transform(x_train)
x_test = preprocesor.fit_transform(x_test)

In [ ]:
y_train["class"] = y_train["class"].astype(int)
y_train= y_train['class'].to_numpy()

y_test["class"] = y_test["class"].astype(int)
y_test= y_test['class'].to_numpy()

<div class="alert alert-block alert-warning"> Tiene que haber una forma Mucho más facil de categorizar y dividir los valores en train y test. Pero si le paso el preprocesor antes, no se como sacar las Y. Aqui separo el df y luego, a categorizo cada uno de los 4 bloques.

In [88]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(2,activation=tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [92]:
model.fit(x_train,
          y_train,
          epochs=8,
          batch_size=64,
          validation_data=(x_test, y_test))

Epoch 1/8
12/12 [==============================] - 0s 11ms/step - loss: 0.0943 - accuracy: 0.9819 - val_loss: 0.1483 - val_accuracy: 0.9625
Epoch 2/8
12/12 [==============================] - 0s 12ms/step - loss: 0.0765 - accuracy: 0.9903 - val_loss: 0.1377 - val_accuracy: 0.9708
Epoch 3/8
12/12 [==============================] - 0s 13ms/step - loss: 0.0621 - accuracy: 0.9930 - val_loss: 0.1279 - val_accuracy: 0.9750
Epoch 4/8
12/12 [==============================] - 0s 15ms/step - loss: 0.0531 - accuracy: 0.9958 - val_loss: 0.1207 - val_accuracy: 0.9708
Epoch 5/8
12/12 [==============================] - 0s 13ms/step - loss: 0.0441 - accuracy: 0.9930 - val_loss: 0.1126 - val_accuracy: 0.9750
Epoch 6/8
12/12 [==============================] - 0s 21ms/step - loss: 0.0365 - accuracy: 0.9958 - val_loss: 0.1050 - val_accuracy: 0.9708
Epoch 7/8
12/12 [==============================] - 0s 13ms/step - loss: 0.0316 - accuracy: 0.9972 - val_loss: 0.1026 - val_accuracy: 0.9792
Epoch 8/8
12/12 [===

In [93]:
val_loss,val_acc = model.evaluate(x_test,y_test)
print(val_loss,val_acc)

8/8 [==============================] - 0s 11ms/step - loss: 0.0919 - accuracy: 0.9708
0.09189478307962418 0.9708333611488342


In [94]:
model.save('tictactoe.h5')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [120]:
full_games = df[working_columns]
full_results = df[target]['class'].to_list()

In [95]:
my_first_model = tf.keras.models.load_model('tictactoe.h5')

In [97]:
prediction = my_first_model.predict(preprocesor.fit_transform(full_games))

In [122]:
np.random.randint(100)

84

In [184]:
for num in range(20):
    n = np.random.randint(958)
    
    if np.argmax(prediction[n]) == 0:
        pred = 'False'
    else:
        pred='True'
    real = full_results[n]
    
    if str(pred) != str(real):
        result='ERROR <<--------------'
    else:
        result='Correct'
    
    print(f'Real: {real} - Predicted: {pred} : {result}')

Real: True - Predicted: True : Correct
Real: False - Predicted: False : Correct
Real: False - Predicted: False : Correct
Real: True - Predicted: True : Correct
Real: True - Predicted: True : Correct
Real: True - Predicted: True : Correct
Real: True - Predicted: True : Correct
Real: False - Predicted: False : Correct
Real: True - Predicted: True : Correct
Real: False - Predicted: True : ERROR <<--------------
Real: False - Predicted: False : Correct
Real: True - Predicted: True : Correct
Real: True - Predicted: True : Correct
Real: True - Predicted: True : Correct
Real: True - Predicted: True : Correct
Real: True - Predicted: True : Correct
Real: False - Predicted: False : Correct
Real: True - Predicted: True : Correct
Real: True - Predicted: True : Correct
Real: True - Predicted: True : Correct


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here